In [1]:
from bw_temporalis import TemporalDistribution as TD, TemporalisLCA
import bw2data as bd
import bw2calc as bc
import numpy as np
import pandas as pd
from functools import partial

In [2]:
bd.projects.set_current("Temporalis example project")

In [3]:
lca = bc.LCA({('temporalis-example', 'EOL'): 1}, ("GWP", "example"))
lca.lci()
lca.lcia()

In [4]:
tlca = TemporalisLCA(lca)

Starting graph traversal
Calculation count: 21


In [5]:
tl = tlca.build_timeline()

In [6]:
tl.build_dataframe()

In [7]:
def supplement_dataframe(df, database_label, columns=["name", "location", "unit", "categories"]):
    database_df = bd.Database(database_label).nodes_to_dataframe()
    database_df['activity'] = database_df['flow'] = database_df['id']
    df = df.join(
        database_df.copy().drop(
            columns=[c for c in database_df.columns if c not in (["flow"] + columns)]
        ).set_index('flow'), 
        on='flow', 
        validate='m:1', 
        rsuffix="_flow"
    )
    df = df.join(
        database_df.copy().drop(
            columns=[c for c in database_df.columns if c not in (["activity"] + columns)]
        ).set_index('activity'), 
        on='activity', 
        validate='m:1', 
        rsuffix="_activity"
    )
    return df.reset_index(drop=True)

In [8]:
supplement_dataframe(tl.df, 'temporalis-example')

,date,amount,flow,activity,name,name_activity
0,2018-05-13,-0.24,1,9,carbon dioxide,Forest
1,2019-05-13,-0.24,1,9,carbon dioxide,Forest
2,2019-05-13,-0.12,1,14,carbon dioxide,Avoided impact - thinnings
3,2022-05-13,-0.24,1,9,carbon dioxide,Forest
4,2022-05-13,-0.12,1,14,carbon dioxide,Avoided impact - thinnings
5,2023-04-27,-0.48,1,13,carbon dioxide,Avoided impact - waste
6,2023-05-13,-0.24,1,9,carbon dioxide,Forest
7,2023-05-13,-0.12,1,14,carbon dioxide,Avoided impact - thinnings
8,2023-07-14,0.10,1,7,carbon dioxide,Production
9,2023-07-14,0.10,1,8,carbon dioxide,Sawmill
